# syntax tools




## locating libraries



### retrieving and framework documentation



### creating libraries from code blocks

Code analysis to combine blocks into modules


### manipulating code libraries

Show git repo organization options?

Warn of changes made outside of the module or feature branch?



## code trees

Functions for working with code trees. selectTree is nice, but jsel and CSS-style selectors don't give you access to parents. True XPath can navigate both directions of an XML/HTML tree. Later in the section selectAst converts the code to an html tree for XPath and CSS-style evaluating.

### tree to html

Reasoning. Every atom is made up of smaller component parts. This conversion creates a tree that breaks down to &lt;elements&gt; and attr="ibutes", just like neutrons and protons. XML, HTML, and CSS parsers are common in every language, which means the XPath and CSS query selectors also might be available.


#### the code

tree to html?


In [ ]:
var importer = require('../Core');
var {selectDom} = importer.import('select tree')
var {JSDOM} = require('jsdom');

function treeToStr(statement, parent) {
    var {type} = statement;
    var result = ``, attrs = ``;
    var isList = true;
    for(var i in statement) {
        if(parseInt(i) + '' === i + '') {
            result += treeToStr(statement[i], parent)
            continue;
        }
        isList = false;
        var jsType = typeof statement[i];
        if(jsType === 'object' && statement[i]) {
            jsType = Object.getPrototypeOf(statement[i]).constructor.name;
        }
        // if the property is an Object, print out as a child
        // TODO: replace this type property when converting other trees
        //   should be like getEl() treeToStr(() => el.type)
        if(statement[i] && typeof statement[i].type !== 'undefined'
           // or if the property is an Array,
           //   print them out as child elements
           || (Array.isArray(statement[i]) && statement[i].length
           && typeof statement[i][0].type !== 'undefined')) {
            // print out the other keys just for fun matching
            attrs +=  `
${i}="" ${i}-type="${jsType}"`;
            result += `
${treeToStr(statement[i], i)}`
            // output as child element and <-attr> elements
            //   for posterity, only child elements are return on the select,
            //   but attr elements can still be matched
        } else {
            // if the property is not an object, i.e. native types
            attrs +=  `
${i}=${JSON.stringify(statement[i])} ${i}-type="${jsType}"`
        }
    }
    
    return isList ? result : `
<${type}${attrs}${parent ? `
 parent-attr="${parent}"`: ``}>${result}</${type}>`
}

function treeToHtml(tree) {
    var body = treeToStr(tree);
    return selectDom('//BODY', '<BODY>' + body + '</BODY>');
}

module.exports = {
    treeToHtml
};


#### justification

Just so it can be parsed with XPath instead of crappy wanna-be query languahes. Get them all working the same!  Also, html/xml parsers are common in many languages. So this library can be parsed and translated to other languages, using itself!




Crap modules I won't need any more:
- css
- ASTQ
- Acorn
- esprima
- escodegen
- esquery
- jsonpath
- jsonpath-plus
- jsonquery
- jspath
- json:select
- jsel
- minimatch

TODO: get rid of jsonpath-plus, jsonstream, jspath, 



#### test converting element to markdown using an ast



```
/* expected output `
[ Script {
    type: 'Program',
    body: [ [FunctionDeclaration], [ExpressionStatement] ],
    sourceType: 'script',
    range: [ 1, 45 ],
    comments: [],
    tokens:

<program type="program">
    <function>
    </function>
    <expression>
    </expression>
</program>

*/
```



### html to tree

Convert HTML back from the function above, html to tree.

#### the code

html to tree?

In [ ]:
var {JSDOM} = require('jsdom');

// TODO: create a pattern same as notebook markdown in core, accumulate
function accumulateChildNodes(body) {
    var commentBuffer = []
    // TODO: exclude children parent properties
    //   left-over children are assigned to children: []
    return Array.from(body.childNodes)
        .reduce((obj, n) => {
            if(n.nodeName === '#text') {
                commentBuffer.push[n];
                return obj;
            }
            var parent = n.getAttribute('parent-attr');
            var newNode = htmlToTree(n);
            if(parent) {
                var parentType = body.getAttribute(parent + '-type')
                if(parentType === 'Array') {
                    if(typeof obj[parent] === 'undefined')
                        obj[parent] = [];
                    obj[parent].push(newNode);
                } else {
                    obj[parent] = newNode;
                }
            } else {
                // TODO: if no children left and no other -attr properties,
                //   remove the child property from the output
                if(typeof obj.children === 'undefined')
                    obj.children = [];
                obj.children.push(newNode);
            }
            if(typeof newNode.comments !== 'undefined') {
                newNode.comments.push.apply(
                    newNode.comments, commentBuffer)
                commentBuffer = [];
            }
            return obj;
        }, {})
}

// expects a string or a tree from JSDOM
function htmlToTree(body) {
    if(typeof body === 'string') {
        var dom = new JSDOM(body);
        return accumulateChildNodes(dom.window.document.body).children;
    }
    if(body.nodeName === '#text') {
        return body.nodeValue;
    }
    // convert attributed object containers back to properties
    // TODO: might have a property named -type and it will be lost
    var attrs = body.getAttributeNames()
        .filter(a => a !== 'parent-attr' && a.substr(-5) != '-type')
        .reduce((obj, p) => {
            var attr = body.getAttribute(p);
            try { obj[p] = JSON.parse(attr); }
            catch (e) { obj[p] = attr; }
            return obj;
        }, {})
    
    var children = accumulateChildNodes(body);
    
    return Object.assign({
        type: body.nodeName
    }, attrs, children);
}

module.exports = {
    htmlToTree
};



#### test tree to html



In [20]:
var importer = require('../Core');
var {treeToHtml} = importer.import('tree to html');
var {htmlToTree} = importer.import('html to tree');

var assert = require('assert');
var esprima = require('esprima'); 

function setupASTArray(code) {
    var ctx = esprima.parse(code, {range: true, tokens: true,
                        comment: true, whitespace: true})
    
    return ctx;
}

function testASTArray(ctx) {
    var page = treeToHtml(ctx);
    var translated_back = htmlToTree(page);
    console.log(translated_back[0]);
    assert(JSON.stringify(ctx).length
           === JSON.stringify(translated_back[0]).length,
           'different code trees');    
    console.log('two JSON trees are same-same')
    return page;
}

module.exports = testASTArray;

function htmlEntities(str) {
    return String(str)
        .replace(/&/g, '&amp;')
        .replace(/</g, '&lt;')
        .replace(/>/g, '&gt;')
        .replace(/"/g, '&quot;');
}

if(typeof $$ !== 'undefined') {
    var code = `
function name(params) {
    
}
console.log()
`
    var ctx = setupASTArray(code);
    var html = testASTArray(ctx);
    $$.html('<pre>' + htmlEntities(html) + '</pre>')
}


compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[5] aka html to tree
{ type: 'Program',
  body:
   [ { type: 'FunctionDeclaration',
       id: [Object],
       params: [Array],
       body: [Object],
       generator: false,
       expression: false,
       async: false,
       range: [Array] },
     { type: 'ExpressionStatement',
       expression: [Object],
       range: [Array] } ],
  sourcetype: 'script',
  range: [ 1, 45 ],
  comments: [],
  tokens:
   [ { type: 'Keyword', value: 'function', range: [Array] },
     { type: 'Identifier', value: 'name', range: [Array] },
     { type: 'Punctuator', value: '(', range: [Array] },
     { type: 'Identifier', value: 'params', range: [Array] },
     { type: 'Punctuator', value: ')', range: [Array] },
     { type: 'Punctuator', value: '{', range: [Array] },
     { type: 'Punctuator', value: '}', range: [Array] },
     { type: 'Identifier', value: 'console', range: [Array] },
     { type: 'Punctuator', value: '.', range: [Array] }

<Program
type="Program" type-type="string"
body="" body-type="Array"
sourceType="script" sourceType-type="string"
range=[1,45] range-type="Array"
comments=[] comments-type="Array"
tokens="" tokens-type="Array">

<FunctionDeclaration
type="FunctionDeclaration" type-type="string"
id="" id-type="Identifier"
params="" params-type="Array"
body="" body-type="BlockStatement"
generator=false generator-type="boolean"
expression=false expression-type="boolean"
async=false async-type="boolean"
range=[1,31] range-type="Array"
 parent-attr="body">

<Identifier
type="Identifier" type-type="string"
name="name" name-type="string"
range=[10,14] range-type="Array"
 parent-attr="id"></Identifier>

<Identifier
type="Identifier" type-type="string"
name="params" name-type="string"
range=[15,21] range-type="Array"
 parent-attr="params"></Identifier>

<BlockStatement
type="BlockStatement" type-type="string"
body=[] body-type="Array"
range=[23,31] range-type="Array"
 parent-attr="body"></BlockStatement></FunctionDeclaration>
<ExpressionStatement
type="ExpressionStatement" type-type="string"
expression="" expression-type="CallExpression"
range=[32,45] range-type="Array"
 parent-attr="body">

<CallExpression
type="CallExpression" type-type="string"
callee="" callee-type="StaticMemberExpression"
arguments=[] arguments-type="Array"
range=[32,45] range-type="Array"
 parent-attr="expression">

<MemberExpression
type="MemberExpression" type-type="string"
computed=false computed-type="boolean"
object="" object-type="Identifier"
property="" property-type="Identifier"
range=[32,43] range-type="Array"
 parent-attr="callee">

<Identifier
type="Identifier" type-type="string"
name="console" name-type="string"
range=[32,39] range-type="Array"
 parent-attr="object"></Identifier>

<Identifier
type="Identifier" type-type="string"
name="log" name-type="string"
range=[40,43] range-type="Array"
 parent-attr="property"></Identifier></MemberExpression></CallExpression></ExpressionStatement>

<Keyword
type="Keyword" type-type="string"
value="function" value-type="string"
range=[1,9] range-type="Array"
 parent-attr="tokens"></Keyword>
<Identifier
type="Identifier" type-type="string"
value="name" value-type="string"
range=[10,14] range-type="Array"
 parent-attr="tokens"></Identifier>
<Punctuator
type="Punctuator" type-type="string"
value="(" value-type="string"
range=[14,15] range-type="Array"
 parent-attr="tokens"></Punctuator>
<Identifier
type="Identifier" type-type="string"
value="params" value-type="string"
range=[15,21] range-type="Array"
 parent-attr="tokens"></Identifier>
<Punctuator
type="Punctuator" type-type="string"
value=")" value-type="string"
range=[21,22] range-type="Array"
 parent-attr="tokens"></Punctuator>
<Punctuator
type="Punctuator" type-type="string"
value="{" value-type="string"
range=[23,24] range-type="Array"
 parent-attr="tokens"></Punctuator>
<Punctuator
type="Punctuator" type-type="string"
value="}" value-type="string"
range=[30,31] range-type="Array"
 parent-attr="tokens"></Punctuator>
<Identifier
type="Identifier" type-type="string"
value="console" value-type="string"
range=[32,39] range-type="Array"
 parent-attr="tokens"></Identifier>
<Punctuator
type="Punctuator" type-type="string"
value="." value-type="string"
range=[39,40] range-type="Array"
 parent-attr="tokens"></Punctuator>
<Identifier
type="Identifier" type-type="string"
value="log" value-type="string"
range=[40,43] range-type="Array"
 parent-attr="tokens"></Identifier>
<Punctuator
type="Punctuator" type-type="string"
value="(" value-type="string"
range=[43,44] range-type="Array"
 parent-attr="tokens"></Punctuator>
<Punctuator
type="Punctuator" type-type="string"
value=")" value-type="string"
range=[44,45] range-type="Array"
 parent-attr="tokens"></Punctuator></Program>

expected output


```
FunctionDeclaration {
    type: 'FunctionDeclaration',
    id:
     Identifier { type: 'Identifier', name: 'name', range: [Array] },
    params: [ [Identifier] ],
    body:
     BlockStatement { type: 'BlockStatement', body: [], range: [Array] },
    generator: false,
    expression: false,
    async: false,
    range: [ 1, 31 ] },
  ExpressionStatement {
    type: 'ExpressionStatement',
    expression:
     CallExpression {
       type: 'CallExpression',
       callee: [StaticMemberExpression],
       arguments: [],
       range: [Array] },
    range: [ 32, 45 ] } ]
```


#### test what happens on regular html converted to a tree




In [ ]:
var importer = require('../Core');
var {htmlToTree} = importer.import('convert html to tree');

function testHtmlTree(page) {
    var translated_back = htmlToTree(page);
    console.log(translated_back[0]);
    return translated_back;
}

if(typeof $$ !== 'undefined') {
    // copied from Chrome Google search homepage
    var code = `<div id="fakebox-container">
      <div id="fakebox">
        <div id="fakebox-search-icon"></div>
        <div id="fakebox-text">Search Google or type a URL</div>
        <input id="fakebox-input" autocomplete="off" tabindex="-1" type="url" aria-hidden="true">
        <div id="fakebox-cursor"></div>
        <button id="fakebox-microphone" title="Search by voice"></button>
      </div>
    </div>`
    var tree = testHtmlTree(code);
    console.log(tree);
}


### select code tree?

select ast?

Select code from tree?

Use CSS selectors to find code.



#### the code


In [8]:
var importer = require('../Core');
var {treeToHtml} = importer.import('tree to html');
var {selectDom, evaluateDom} = importer.import('select tree')
var esprima = require('esprima'); 

function selectAst(descriptor, code) {
    if(typeof code === 'string') {
        code = esprima.parse(code, {range: true, tokens: true,
                                  comment: true, whitespace: true});
    }
    
    // TODO: move this conversion to selectDom for all objects?
    //   No more selectTree because jsel and friends are inconsistent.
    if(typeof code.evaluate === 'undefined'
       && typeof code.ownerDocument === 'undefined')
        code = treeToHtml(code);
    
    return selectDom(descriptor, code);
}

module.exports = {
    selectAst
}


compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[3] aka tree to html


{ walkAst: [Function: walkAst],
  selectAst: [Function: selectAst] }

#### test selecting xpath with parent data

Test selecting trees with the html translator.

Get the same result as using the json evaluator.


In [1]:
var importer = require('../Core');
//var {makeXpath} = importer.import('make long xpath');
var {selectAst} = importer.import('select code tree')
var {selectDom} = importer.import('select tree')
var {makeXpath} = importer.import('make long xpath')
var {htmlToTree} = importer.import('html to tree')
var escodegen = require('escodegen');
var assert = require('assert');

var code = `
var importer = require('../Core');
function name(params) {
    return importer.interpret('this is a describe request');
}
console.log()
`

function testCodeToDom(code) {
    // make a path with the interpret symbol
    var node1 = selectAst(`//*[@name="interpret"]`, code)
    console.log(node1)
    var parent = selectDom(`parent::*`, node1);
    assert(parent.nodeName !== '', 'parent selector works');
    console.log(selectDom(`*`, node1).children.length);
    console.log(htmlToTree(parent));
}
// TODO: compare with acorn
if(typeof $$ !== 'undefined') {
    testCodeToDom(code);
}


compiling /Users/briancullinan/jupyter_ops/Core/files.ipynb[0] aka list project files
compiling /Users/briancullinan/jupyter_ops/Core/files.ipynb[1] aka common ignore paths
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[8] aka select ast
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[3] aka tree to html
compiling /Users/briancullinan/jupyter_ops/Core/patterns.ipynb[2] aka ### select tree
compiling /Users/briancullinan/jupyter_ops/Core/patterns.ipynb[0] aka ,### walk tree
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[10] aka ,#### make long xpath
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[5] aka html to tree
Script {
  type: 'Program',
  body:
   [ VariableDeclaration {
       type: 'VariableDeclaration',
       declarations: [Array],
       kind: 'var',
       range: [Array] },
     FunctionDeclaration {
       type: 'FunctionDeclaration',
       id: [Identifier],
       params: [Array],
       body: [BlockStatement],
       

#### TODO: test with acorn instead


### get xpath to node?

XPath minimizing service from recording prototype app.
 

#### make long xpath?

For generating links to nodes from the node, uses parentElement.



In [ ]:
var importer = require('../Core');
var {selectDom} = importer.import('select tree');
var assert = require('assert');
// algorithm magic!
// create an very specific XPath following these rules

function makeXpath(node) {
    var classifiers = [{
        tag: `local-name(.)`,
        ids: [`@*[name()="type" or name()="id" or name()="name"]`],
        classNames: `@class`,
        index: `count(preceding-sibling::*)`,
        parent: `local-name(parent::*)`,
        //parent: `parent::*/@*[name()="type" or name()="id" or name()="name"]`,
        nthType: `count(preceding-sibling::*[local-name(*)=local-name(.)])`
    }, ({tag, ids, classNames, index, parent, nthType}) => ({
        tag: tag && tag !== 'object' ? tag : `*`,
        id: ids.length
            ? `[${ids
            .map(id => 
                 `(@id="${id}" or @name="${id}" or @type="${id}")`)
            .join(` and `)}]`
            : ``,
        classNames: classNames.length > 0
            ? `[contains(@class, "${classNames}")]`
            : ``,
        index: typeof index !== 'undefined' && parent
            ? `/parent::${parent}/*[${index+1}]`
            : ``,
        parent: parent && typeof nthType !== 'undefined'
            ? `/parent::${parent}/${
                tag && tag !== 'object' ? tag : `*`}[${nthType+1}]`
            : ``
        
    })]
    
    var match = selectDom([
        classifiers,
        ({tag, id, classNames, index, parent}) =>
            `${tag}${id}${classNames}${index}${parent}`
    ], node)[0];
    assert(selectDom(match, node),
           'oops, the xpath expression maker messed up');
    return match;
}

module.exports = {
    makeXpath
};


#### test making an xpath from any arbitraty node

generate css selectors from code?

problem, jsel doesn't support parent()


In [3]:
var esprima = require('esprima'); 
var escodegen = require('escodegen');
var assert = require('assert');
var importer = require('../Core');
var {selectAst} = importer.import('select code tree')
var {selectDom} = importer.import('select tree')
var {makeXpath} = importer.import('make long xpath')
var {htmlToTree} = importer.import('html to tree')

var code = `
var importer = require('../Core');
function name(params) {
    return importer.interpret('this is a describe request');
}
console.log()
`

function testMakeXpath(code) {
    // make a path with the interpret symbol
    var node1 = selectAst(`//*[@name="name"]`, code)
    var parent = selectDom(`//*[@type="FunctionDeclaration"]`, node1);
    var output = makeXpath(parent)
    var node2 = selectAst(`//${output}`, code)
    console.log(node2)
    assert(node1.parentNode.getAttribute('name') === node2.getAttribute('name'));
    return node2
    
}
// TODO: compare with acorn
if(typeof $$ !== 'undefined') {
    var nodes = testMakeXpath(code);
    console.log(htmlToTree(nodes));
}


compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[8] aka select ast
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[3] aka tree to html
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[10] aka ,#### make long xpath
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[5] aka html to tree
HTMLUnknownElement {
  [Symbol(SameObject caches)]:
   [Object: null prototype] {
     childNodes:
      NodeList {
        '0': [Text],
        '1': [HTMLUnknownElement],
        '2': [Text],
        '3': [HTMLUnknownElement],
        '4': [Text],
        '5': [HTMLUnknownElement] },
     children:
      HTMLCollection {
        '0': [HTMLUnknownElement],
        '1': [HTMLUnknownElement],
        '2': [HTMLUnknownElement] } } }
{ type: 'FunctionDeclaration',
  id: { type: 'Identifier', name: 'name', range: [ 45, 49 ] },
  params: [ { type: 'Identifier', name: 'params', range: [Array] } ],
  body:
   { type: 'BlockStatement',
     body: [ [Object] ],
     ran

#### minimize xpath



In [ ]:

function minimizeXpath(fullPath) {
    // flatten the XPath stack using '/'
    const fullPath = '//*[not(./app-root)]//' + pathStack.join('/');
    const matches = RecordingComponent.getArrayXPath(fullPath);
    if (matches.length !== 1) {
        throw new Error('Can\'t do anything right!');
    }

    // minimize XPath by splitting up every combination
    //   and checking only 1 element is matched for click
    const combinations: Array<string> = [];
    // * between 1 to N segments
    for (let i = 1; i < pathStack.length; i++) {
        for (let j = 1; j < i; j++) {
            const segments = ([
                ...pathStack.slice(j, i),
                // replace match with an extra / to create // 
                //   between unnecessary segments
                '', // added to create // between target element
                pathStack[ pathStack.length - 1 ]
            ]).join('/');
            combinations.push('//' + segments);
        }
        // * starts from 0 to N
        // *
    }

    // sort by smallest and return shortest path matching 1 DOM element
    const minimal = combinations
            .filter(c => RecordingComponent.getArrayXPath(c).length === 1)
            .sort((a, b) => a.length - b.length);

    return minimal[ 0 ];

    // TODO: prioritize by button/input, ids, classes, attributes? (extra credit), index
}

module.exports = minimizeXpath;


#### convert simple xpath to css selector

In [ ]:

static convertXPathToCss(path: string): string {
    // if matched xpath is simple enough, convert it to CSS
    // DIV[contains(@class, "product-tile")]/parent::*/DIV[2]//MD-CARD[contains(@class, "mat-card")]/parent::*/MD-CARD[1]
    return path
            .replace(/\/([a-z-]+)\[@id="(.*?)"]/ig, '/$1#$2')
            .replace(/\/([^\/]+)\[contains\(@class, "(.*?)"\)]/ig, '/$1.$2')
            .replace(/\/parent::[a-z-]+\/[a-z-]+\[([0-9]+)]/ig, ':nth-child($1)')
            .replace(/^\/\//ig, '')
            .replace(/\/\//ig, ' ')
            .replace(/\//ig, ' > ');
}



### improved syntax manipulation

Replacing the functionality I noticed in babel somewhere. Basically walking a tree with visitors is ugly, so we replace it with a sequence of CSS style queries.



#### select expression?

select from code?

Convert the body of a function to a list of search strings.


In [ ]:
var importer = require('../Core');
var {selectAst} = importer.import('select ast');
var {selectDom} = importer.import('select tree');
var {makeXpath} = importer.import('make long xpath')

// create an XPath search out of the function body
//   using the parameters as matching predicates
function selectExpr(code) {
    var func = selectAst([
        `(*/FunctionDeclaration|*/*/ArrowFunctionExpression)`
    ], code.toString())[0]
    var expression = selectDom([
        `Identifier[@parent-attr="params"]/@name`,
        // select all elements from func that match param name
        //   TODO: default functionality can be overridden by specifying
        //   a function as the default parameter value
        (name) => selectDom(`BlockStatement//*[@name="${name}"]`, func)
    ], func)[0]; // TODO: make multiple expressions?
    var xpath = makeXpath(expression);
    
    // TODO: add minimizing expressions, unique to the original context
    return xpath;
}

module.exports = {
    selectExpr
}


#### test selecting expressions


In [ ]:
var importer = require('../Core');
var {selectExpr} = importer.import('select expression');

if(typeof $$ !== 'undefined') {
    var xpath = selectExpr((importer) => {
            var importer = require('../Core');
        });
    
    console.log(xpath)
//    replaceCore(code)
}



#### test created expression on notebook code?


In [ ]:
var importer = require('../Core');
var {selectExpr} = importer.import('select expression');
var {selectAst} = importer.import('select ast');

if(typeof $$ !== 'undefined') {
    var xpath = selectExpr((importer) => {
            var importer = require('../Core');
        });
    var allCells = importer.interpret([
        'languages.ipynb[test created expression]'
    ]);
    console.log(allCells.filter(cell => {
        try {
            var match = selectAst([`//${xpath}`], cell.code);
            return match.length > 0;
        } catch (e) {
            return false;
        }
    }))
    console.log(xpath)
//    replaceCore(code)
}


### TODO: generate unit tests

TODO: use language server for this

generate unit test from logic branching?


In [ ]:
var acorn = require("acorn");
var ASTQ  = require("astq");
var _ = require('lodash');
var importer = require('../Core');

var source = importer.interpret('zuora eloqua mapper').code;
    
var comments = [], tokens = [];

var ast = acorn.parse(source, {
    ecmaVersion: 6,
    // collect ranges for each node
	ranges: true,
	// collect comments in Esprima's format
	onComment: comments,
	// collect token ranges
	onToken: tokens,
    
})

var astq = new ASTQ();
var assignments = astq.query(ast, `
//AssignmentExpression [
    //Identifier[@name=='record']
]
`);
var grouped = _.groupBy(assignments, a => comments.filter(c => c.start < a.start).pop().value.trim());
var output = '';
Object.keys(grouped).forEach(k => {
    const ass = grouped[k].map(as => {
        const leftMembers = as.left.type === 'MemberExpression'
            ? [as.left.property.value] : astq.query(as.left, `
//MemberExpression
`).map(m => m.property.value).filter(m => m);
        const rightMembers = as.right.type === 'MemberExpression'
            ? [as.right.property.value] : astq.query(as.right, `
//MemberExpression
`).map(m => m.property.value).filter(m => m);
        if(leftMembers.length === 0 || rightMembers.length === 0) {
            return;
        }
        return `
            it('should map ${leftMembers.join(', ')} to eloqua given ${rightMembers.join(', ')} from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['${rightMembers.join('\'] || records[0][\'')}'], mapped[0]['${leftMembers}']);
            })
`;
    }).filter(m => m).join('\n');
    output += `
        describe('${k} properties mapped', () => {
            ${ass}
        });
`
})
//console.log(astq.query(ast, `//AssignmentExpression`));
var destination = [].concat(...astq.query(ast, `//AssignmentExpression`)
    .map(as => as.left.type === 'MemberExpression'
            ? [as.left.property.value]
            : astq.query(as.left, `//MemberExpression`)
         .map(m => m.property.value)
         )).filter(m => m)
var source = astq.query(ast, `
//MemberExpression [
    //Literal
]
`)
    .filter(e => (e.property.value + '').includes('.'))
    .map(e => e.property.value)
console.log(`
var assert = require('assert');
var importer = require('../Core');
var renewalsQuery = importer.import('zuora renewals query');
var { bulkImportTemplate } = importer.import('eloqua create template');
var { getUniqueRatePlans } = importer.import('zuora account blueprints');

var zuoraQuery = renewalsQuery.getQuery('beginning of November', 'beginning of December').Query;

var accounts = getUniqueRatePlans();
for(var records of accounts) {

    describe('given account with ' + records.map(r => r['ProductRatePlan.Name']), () => {
        ${output}
    })

}

describe('check field map', () => {
    it('should map all fields from zuora query', () => {
        const requireFields = ${JSON.stringify(source, null, 4).replace(/\n/ig, '\n    ')};
        const missing = requireFields.filter(f => !zuoraQuery.includes(f));
        assert(missing.length === 0, 'missing fields from zuora query ' + missing.join(', '));
    })

    it('should map all fields in eloqua import definition', () => {
        const mappedFields = ${JSON.stringify(destination, null, 4).replace(/\n/ig, '\n    ')};
        const unmapped = Object.keys(bulkImportTemplate(60).fields).filter(k => !mappedFields.includes(k));
        assert(unmapped.length === 0, 'unmapped eloqua fields ' + unmapped.join(', '));
    })

    it('should map all record fields to eloqua fields', () => {
        const mappedFields = ${JSON.stringify(destination, null, 4).replace(/\n/ig, '\n    ')};
        const missing = mappedFields.filter(f => !Object.keys(bulkImportTemplate(60).fields).includes(f));
        assert(missing.length === 0, 'missing fields from eloqua import definition ' + missing.join(', '));
    })
})

`);


### TODO: detecting framework features

Show router tree, module import tree, call tree, feature tree


### TODO: format detection by example

Lint all code

Reformat all code like when using webstorm


### common tree queries

TODO: remove all of these and replace with new transpiler


common astq and esquery for notebooks?

common ast queries?


In [ ]:
var GET_CORE = `/VariableDeclaration[//Literal[index(@value, 'Core') > 0]]`;
var GET_CORENAME = `/*/VariableDeclarator/Identifier`;
var GET_DECLARATIONS = `//*[@type == 'VariableDeclarator'
|| @type == 'FunctionDeclaration'
|| @type == 'StaticMemberExpression']`;
var GET_DECLARED = fnName => `${GET_DECLARATIONS}/Identifier[@name == '${fnName}']`;
var GET_CALLED = fnName => `//CallExpression/Identifier[@name == '${fnName}']`;
var GET_IMPORTER = importer => `/*[(@type == 'StaticMemberExpression' || @type == 'MemberExpression')
                && /Identifier[@name == '${importer}']]`;
var GET_CALLER = importer => `//CallExpression[${GET_IMPORTER(importer)}]`;
var GET_PARENT = of => `//*[${of}]`;
// TODO: use form code analysis
var GET_IMPORT_IDENTIFIER = `/Identifier[@name=='import' || @name=='importNotebook']`;
var GET_IMPORT = match => `//CallExpression[
    (${GET_IMPORT_IDENTIFIER}
    || /StaticMemberExpression${GET_IMPORT_IDENTIFIER})
    && //Literal[@value == '${match}']]`;
var GET_EXPORTS = `//AssignmentExpression[//Identifier[@name=='exports']]`;
var GET_FUNCTIONS = `//FunctionDeclaration`;
var GET_IMPORTS = `//CallExpression[${GET_IMPORT_IDENTIFIER} 
    || /StaticMemberExpression${GET_IMPORT_IDENTIFIER}]//Literal`;
var GET_IMPORTER_CALLS = `//CallExpression//Identifier[
    ${[ 'import',
  'streamJson',
  'regexToArray',
  'getCells',
  'runAllPromises',
  'interpret',
  'runInNewContext',
  'cacheCells',
  'interpretMarkdown',
  'fuseSearch',
  'cacheAll' ].map(k => `@name=='${k}'`).join(' || ')}]`
var GET_PARAMETERS = fnName => `//FunctionDeclaration[
    /Identifier[@name == '${fnName}']
]/*[@type != 'BlockStatement']//Identifier`;
var GET_REQUIRE = `//CallExpression[/Identifier[@name=='require']]//Literal`;

module.exports = {
    GET_CORE,
    GET_CORENAME,
    GET_DECLARATIONS,
    GET_DECLARED,
    GET_CALLED,
    GET_IMPORTER,
    GET_CALLER,
    GET_PARENT,
    GET_IMPORT_IDENTIFIER,
    GET_IMPORT,
    GET_EXPORTS,
    GET_FUNCTIONS,
    GET_IMPORTS,
    GET_IMPORTER_CALLS,
    GET_PARAMETERS,
    GET_REQUIRE
}


get ast path array?

TODO: make this act more like getArrayXPath but without dependence on astq, esquery, and esprima




In [ ]:
var ASTQ  = require("astq");
var astq = new ASTQ();
var esquery = require('esquery');
var esprima = require('esprima'); 

function getArrayAST(selector, ctx, index) {
    if(typeof ctx === 'string') {
        ctx = esprima.parse(ctx, {range: true, tokens: true,
                                  comment: true, whitespace: true});
    }
    try {
        return astq.query(ctx, selector);
    } catch (e) {
        if(e.message.includes('query parsing failed')) {
            return esquery(ctx, selector);
        } else {
            throw e;
        }
    }
}

// TODO: implement getObjectXPath with ASTs

module.exports = getArrayAST;



test ast path array?

In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');

function testASTArray(selector, search = '') {
    return getArrayAST(selector, importer.interpret(search).code);
}

module.exports = testASTArray;



get module exports from source?

get exports from source?


In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var {GET_EXPORTS, GET_FUNCTIONS} = importer.import('common ast queries');

// TODO: does this work on other node_modules?
function getExports(code) {
    // can't call it exports or it might detect itself
    var moduleOutput = getArrayAST(GET_EXPORTS, code)
    var exportFunctions = [].concat.apply([], moduleOutput
                                          .map(e => getArrayAST('/Identifier', e)
                                               .map(e => e.name)));
    var outputFunctions = getArrayAST(GET_FUNCTIONS, code)
        .filter(f => exportFunctions.includes(f.id.name))
        .map(f => f.id.name)
    var assignments = moduleOutput
        .map(e => getArrayAST('/StaticMemberExpression', e)
             .map(e => e.property.name))
    return outputFunctions.concat.apply(outputFunctions, assignments);
}

module.exports = getExports;


get parameter names?

function parameters?


In [ ]:
var importer = require('../Core');
var getExports = importer.import('get module exports');
var getArrayAST = importer.import('get ast path array');
var {GET_PARAMETERS, GET_FUNCTIONS} = importer.import('common ast queries');

// TODO: fix calls to this and remove interpret
function getParameters(search) {
    const result = typeof search === 'function'
        ? search.toString()
        : typeof search === 'string'
            ? importer.interpret(search).code
            : search;
    
    var fnName = getExports(result)[0]
        || getArrayAST(GET_FUNCTIONS, result).map(f => f.id.name)[0];
    if(!fnName) {
        return [];
    }
    
    var parameters = getArrayAST(GET_PARAMETERS(fnName), result).map(p => p.name);
    return [fnName, ...(parameters || [])];
}

module.exports = getParameters;


get imports from source?

In [ ]:
var importer = require('../Core');
var importerCode = importer.interpret('import.ipynb');
var getExports = importer.import('get module exports');
var getArrayAST = importer.import('get ast path array');
var {GET_IMPORTS, GET_IMPORTER_CALLS} = importer.import('common ast queries');

function getImports(code) {
    var imports = getArrayAST(GET_IMPORTS, code)
        .map(s => s.value);
    var coreMatches = getArrayAST(GET_IMPORTER_CALLS, code)
        .map(s => s.name)
        .map(i => importerCode
             .filter(core => (core.exports = core.exports || getExports(core.code))
                     .includes(i))[0].id);
    return imports.concat(coreMatches);
}

module.exports = getImports;



get requires from source?

In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var {GET_REQUIRE} = importer.import('common ast queries');

function getRequires(code) {
    return getArrayAST(GET_REQUIRE, code)
        .map(s => s.value)
}


module.exports = getRequires;



get all ast properties?

In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var getExports = importer.import('get exports from source');
var getParameters = importer.import('function parameters');
var getImports = importer.import('get imports from source');

function addAST(cell) {
    cell = typeof cell === 'string'
            ? importer.interpret(cell)
            : cell;
    var code = cell.code || cell;
    try {
        cell.ast = getArrayAST('*', code);
    } catch (e) {
        return cell;
    }
    return Object.assign(cell, {
        exports: getExports(cell.ast),
        params: getParameters(cell.ast),
        imports: getImports(cell.ast)
    });
}

module.exports = addAST;

